In [1]:
import os
import re
import pandas as pd

# Pfad zum Hauptverzeichnis, in dem die Unterverzeichnisse liegen
BASE_DIR = "data/liboqs"

def extract_model_name(directory_name):
    """Extrahiert den Modelnamen aus dem Verzeichnisnamen."""
    match = re.match(r"testresult_liboqs_(.*?)_\d{4}\.\d{2}\.\d{2}_\d{2}\.\d{2}", directory_name)
    return match.group(1) if match else directory_name

def parse_liboqs_kem_speed_log(log_path, model_name):
    """Liest die log-Datei und gibt eine Liste mit Ergebnissen zurück."""
    if not os.path.isfile(log_path):
        return []

    with open(log_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    data = []
    current_algorithm = None

    for line in lines:
        parts = [p.strip() for p in line.split("|")]
        if len(parts) < 7:
            continue

        op, iters_str, _, _, _, highprec_str, _ = parts

        if op and not iters_str:
            current_algorithm = op
        elif op in ("keygen", "encaps", "decaps") and iters_str.isdigit() and highprec_str.isdigit():
            rounds = int(iters_str)
            ms_mean = round(float(highprec_str) / 1_000_000.0, 3)
            data.append((model_name, current_algorithm, op, rounds, ms_mean))

    from collections import defaultdict
    grouped = defaultdict(dict)
    for m, alg, op, rnd, ms in data:
        grouped[(m, alg)][op] = (rnd, ms)

    records = []
    for (m, alg), ops_dict in grouped.items():
        if all(k in ops_dict for k in ["keygen", "encaps", "decaps"]):
            k_r, k_ms = ops_dict["keygen"]
            e_r, e_ms = ops_dict["encaps"]
            d_r, d_ms = ops_dict["decaps"]

            total_ms = round(k_ms + e_ms + d_ms, 3)

            records.append({
                "Model": m,
                "Algorithmus": alg,
                "Runden_key": k_r,
                "kem_keypair (ms)": k_ms,
                "Runden_enc": e_r,
                "kem_enc (ms)": e_ms,
                "Runden_dec": d_r,
                "kem_dec (ms)": d_ms,
                "Total Time (ms)": total_ms
            })

    return records

all_records = []
for directory in os.listdir(BASE_DIR):
    full_path = os.path.join(BASE_DIR, directory)
    if os.path.isdir(full_path):
        log_file = os.path.join(full_path, "liboqs_kem_speed.log")
        if os.path.isfile(log_file):
            model_name = extract_model_name(directory)
            recs = parse_liboqs_kem_speed_log(log_file, model_name)
            all_records.extend(recs)

df = pd.DataFrame(all_records)
df = df[["Model", "Algorithmus", "Runden_key", "kem_keypair (ms)", "Runden_enc", "kem_enc (ms)", "Runden_dec", "kem_dec (ms)", "Total Time (ms)"]]
df = df.sort_values(by=["Algorithmus", "Model"])

# Speichere als CSV
df.to_csv("output/csv/liboqs_time_kem.csv", index=False)
#df.to_excel("liboqs.time.mittelwert.xlsx", index=False)

# Anzeige im Jupyter Notebook
#from IPython.display import display
#print("")
#display(df)
df_formatted = df.copy()
df_formatted[df_formatted.select_dtypes(include=['float64', 'float32']).columns] = (
    df_formatted.select_dtypes(include=['float64', 'float32']).map(lambda x: f"{x:.3f}".replace('.', ','))
)
#display(df_formatted)
df_formatted


,Model,Algorithmus,Runden_key,kem_keypair (ms),Runden_enc,kem_enc (ms),Runden_dec,kem_dec (ms),Total Time (ms)
52,Pi3b,BIKE-L1,16,"199,012",274,"10,974",18,"171,737","381,723"
81,Pi4b,BIKE-L1,55,"55,047",1085,"2,767",70,"42,905","100,719"
136,PiZero2,BIKE-L1,147,"20,483",1403,"2,139",84,"36,101","58,723"
53,Pi3b,BIKE-L3,5,"614,293",90,"33,410",6,"538,251","1185,954"
82,Pi4b,BIKE-L3,18,"166,720",355,"8,452",23,"134,696","309,868"
...,...,...,...,...,...,...,...,...,...
74,Pi3b,sntrup761,57,"53,137",1520,"1,975",774,"3,877","58,989"
103,Pi4b,sntrup761,227,"13,253",6780,"0,442",3611,"0,831","14,526"
129,PiBanana,sntrup761,27,"111,805",887,"3,385",286,"10,521","125,711"
181,PiZero,sntrup761,12,"256,338",644,"4,655",189,"15,950","276,943"
